In [6]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools
import csv
import pandas as pd
import io
import checklist.editor
import checklist.text_generation
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb
from spacy.cli import download
download("en_core_web_sm")
import numpy as np
import re


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0-py3-none-any.whl (13.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
# read test data
df = pd.read_csv('testdataset.txt', sep='\t', header=None)
df.columns = ['labels', 'text_a', 'text_b', 'id', 'stance_labels']
topic_sentences = df['text_a'].tolist()
claim_sentences = df['text_b'].tolist()
stances = df['stance_labels'].tolist()

7065


In [13]:
# checklist function to replace pairs of words
def replace_pairs(pairs):
    def replace_z(text):
        ret = []
        for x, y in pairs:
            t = re.sub(r'\b%s\b' % x, y, text )
            if t != text:
                ret.append(t)
            if y == 'smart':
                continue
            t = re.sub(r'\b%s\b' % y, x, text )
            if t != text:
                ret.append(t)
        return list(set(ret))
    return replace_z
def apply_and_pair(fn):
    def ret_fn(text):
        ret = fn(text)
        return [(r) for r in ret]
    return ret_fn

In [15]:
# list of words and their synonyms
synonyms = [('restrict', 'limit'),('support', 'back'), ('prohibited', 'banned'), ('prohibit', 'ban'), ('promote', 'encourage'), ('mandatory', 'compulsory'), ('legalize', 'authorize'), ('embrace', 'accept'), ('subsidize', 'fund'), ('cancel', 'abolish'), ('replace', 'substitute'), ('curb', 'constrain'), ('Negative', 'adverse'), ('eliminate', 'discard'), ('stop', 'cease'), ('provide', 'give'), ('abolished', 'eradicated'), ('reflect', 'display'), ('children', 'kids'), ('abandon', 'discard'), ('protect', 'safeguard'), ('free', 'nothing'), ('allowed', 'welcomed'), ('advocate', 'endorse'), ('block', 'obstruct'), ('sympathize with', 'appreciate'), ('harmful', 'damaging'), ('remain', 'stay'), ('bring', 'create'), ('fight for', 'defend'), ('implement', 'execute'), ('return', 'change'), ('forced', 'required')]
antonyms = [('restrict', 'allow'), ('limit', 'encourage'), ('support', 'oppose'), ('prohibited', 'permitted'), ('prohibit', 'permit'), ('promote', 'impede'), ('compulsory', 'optional'), ('mandatory', 'discretional'), ('legalize', 'forbid'), ('accept', 'refuse'), ('embrace', 'reject'), ('subsidize', 'reject'), ('cancel', 'keep'), ('banned', 'legitimate'), ('replace', 'not replace'), ('curb', 'encourage'), ('Negative', 'positive'), ('eliminate', 'preserve'), ('stop', 'continue'), ('provide', 'not provide'), ('abolished', 'kept'), ('ban', 'legitimatize'),('reflect', 'not reflect'), ('children', 'adults'), ('abandon', 'retain'), ('protect', 'neglect'), ('free', 'a cost'), ('allowed', 'not allowed'), ('advocate', 'deny'), ('block', 'help'), ('sympathize with', 'not sympathize with'), ('harmful', 'beneficial'), ('remain', 'not remain'), ('bring', 'stop'), ('fight for', 'not fight for'), ('implement', 'not implement'), ('fund', 'not fund'),('return', 'not return'), ('forced', 'unforced'), ('stay away from', 'engage with')]

In [16]:
nlp = spacy.load('en_core_web_sm')
pdata = list(nlp.pipe(topic_sentences))

In [17]:
# replace verbs in topic sentence with synonyms
ret = Perturb.perturb(topic_sentences, replace_pairs(synonyms), keep_original=True)
synonym_list = ret.data

In [18]:
# replace verbs in topic sentence with antonyms
ret2 = Perturb.perturb(topic_sentences, replace_pairs(antonyms), keep_original=True)
antonym_list = ret2.data

In [19]:
# replace contractions with expansion and contractable words with contractions
ret3 = Perturb.perturb(claim_sentences, Perturb.contractions, keep_original=True)
contraction_list = ret3.data

In [20]:
# change location names in sentences to random locations
nlp = spacy.load('en_core_web_sm')
pdata = list(nlp.pipe(claim_sentences))
ret4 = Perturb.perturb(pdata, Perturb.change_location, n=1, keep_original=True)
location_list = ret4.data
print(location_list[:10])

[['The Seattle-based Great Ape Project (GAP), founded by Peter Singer, is campaigning for the United Nations to adopt its Declaration on Great Apes, which would see chimpanzees, bonobos, gorillas and orang-utans included in a community of equals with human beings.', 'The Wichita-based Great Ape Project (GAP), founded by Peter Singer, is campaigning for the United Nations to adopt its Declaration on Great Apes, which would see chimpanzees, bonobos, gorillas and orang-utans included in a community of equals with human beings.'], ['In 1992, Switzerland amended its constitution to recognize animals as beings and not things [ref].', 'In 1992, Thailand amended its constitution to recognize animals as beings and not things [ref].'], ['A decade later, Germany guaranteed rights to animals in a 2002 amendment to its constitution, becoming the first European Union member to do so [ref].', 'A decade later, Russian Federation guaranteed rights to animals in a 2002 amendment to its constitution, bec

In [21]:
# change numbers in sentences to random numbers
pdata2 = list(nlp.pipe(claim_sentences))
ret5 = Perturb.perturb(pdata2, Perturb.change_number, n=1, keep_original=True)
number_list = ret5.data

[["Other early influences include: Upton Sinclair's 1906 novel The Jungle, which drew attention to obscured slaughterhouse operations; Henry Salt's treatises on nonhuman animal rights; which drew from human abolitionist arguments for recognizing personhood of people considered to be property; and the Alcott House of New England, a community serving as a stop on the Underground Railroad and requiring its residents to eat a vegan diet [ref].", "Other early influences include: Upton Sinclair's 2017 novel The Jungle, which drew attention to obscured slaughterhouse operations; Henry Salt's treatises on nonhuman animal rights; which drew from human abolitionist arguments for recognizing personhood of people considered to be property; and the Alcott House of New England, a community serving as a stop on the Underground Railroad and requiring its residents to eat a vegan diet [ref]."], ['It was a 1965 article by novelist Brigid Brophy in The Sunday Times which was pivotal in helping to spark t